In [1]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tempfile as tmp
import soundfile as sf
from scipy import signal, stats
from scipy.io import wavfile
from typing import Tuple, Union
from sklearn import model_selection, preprocessing
from tensorflow import keras
from tensorflow.keras import layers, optimizers, utils
from tensorflow_addons import activations as tfa_activations
from keras_nlp import layers as nlp_layers

In [2]:
def trim_wav_file(path: str, st: int = 0, ed: int = 10) -> np.ndarray:
    (sampling_rate, wave_data) = wavfile.read(filename=path)
    st_sample: int = int(st * sampling_rate)
    ed_sample: int = int(ed * sampling_rate)
    wave_data = wave_data[st_sample:ed_sample]
    del st_sample, ed_sample
    return wave_data

def get_signal_list(path: str) -> list[np.ndarray]:
    file_names: list[str] = glob.glob(pathname=f"{path}/*.wav")
    sample_list: list[np.ndarray] = []
    for file_name in file_names:
        sample: np.ndarray = trim_wav_file(path=file_name, st=0, ed=10)
        sample_list.append(sample)
        del sample
    del file_name, file_names
    return sample_list

def combine_signal_list(signal_list: list[np.ndarray], mn_len: int) -> np.ndarray:
    sample_list: list[np.ndarray] = []
    for sgnal in signal_list:
        sample_list.append(sgnal[:mn_len])
    del sgnal
    return np.stack(sample_list, axis=0)

orig_list: list[np.ndarray] = get_signal_list(path="Dataset/Signal")
noise_list: list[np.ndarray] = get_signal_list(path="Dataset/Noise")

mn: int = 1e9
for orig in orig_list:
    mn = min(mn, orig.shape[0])
del orig
for noise in noise_list:
    mn = min(mn, noise.shape[0])
del noise

orig: np.ndarray = combine_signal_list(orig_list, mn)
noise: np.ndarray = combine_signal_list(noise_list, mn)[:, :, 0]
del mn, orig_list, noise_list

C:\Users\Matt\AppData\Local\Temp\ipykernel_3048\2002002543.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  (sampling_rate, wave_data) = wavfile.read(filename=path)


In [3]:
orig = signal.resample_poly(orig, up=1, down=3, axis=1)
noise = signal.resample(noise, num=1898, axis=1)

In [4]:
def gen_x_y_div(orig: np.ndarray, noise: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    x_list: list[np.ndarray] = []
    y_list: list[np.ndarray] = []
    x_avg_list: list[np.ndarray] = []
    x_sd_list: list[np.ndarray] = []
    for i in range(0, orig.shape[0]):
        for j in range(0, noise.shape[0]):
            combi: np.ndarray = orig[i] + noise[j]
            x_list.append(np.stack([combi, noise[j]], axis=1))
            x_avg_list.append(np.stack([np.average(combi, axis=0), np.average(noise[j], axis=0)], axis=0)[np.newaxis, :])
            x_sd_list.append(np.stack([np.std(combi, axis=0), np.std(noise[j], axis=0)], axis=0)[np.newaxis, :])
            y_list.append(orig[i])
            del combi
        del j
    del i
    x: np.ndarray = np.stack(x_list, axis=0)
    y: np.ndarray = np.stack(y_list, axis=0)
    x_avg: np.ndarray = np.stack(x_avg_list, axis=0)
    x_sd: np.ndarray = np.stack(x_sd_list, axis=0)
    return (x, y, x_avg, x_sd)

(x, y, x_avg, x_sd) = gen_x_y_div(orig, noise)
x = (x - x_avg) / x_sd
y = (y - x_avg[:, :, 0]) / x_sd[:, :, 0]
y = y[:, :, np.newaxis]

In [5]:
(x_train, x_test, y_train, y_test) = model_selection.train_test_split(x, y, test_size=0.2)

In [6]:
def build_trans() -> keras.Model:
    inputs = keras.Input(x.shape[1:])
    m = inputs
    emb = nlp_layers.PositionEmbedding(sequence_length=x.shape[1])(m)
    m = layers.Add()([m, emb])
    m = nlp_layers.TransformerEncoder(intermediate_dim=2, num_heads=2, dropout=0, activation="relu")(m)
    m = nlp_layers.TransformerDecoder(intermediate_dim=2, num_heads=2, dropout=0, activation="relu")(m)
    m = layers.Dense(units=2, activation=tfa_activations.mish)(m)
    m = layers.Dense(units=2, activation=tfa_activations.mish)(m)
    m = layers.Dense(units=1, activation="linear")(m)
    outputs = m
    base_model = keras.Model(inputs, outputs)
    custom_objs: dict = {
        "keras_nlp>PositionEmbedding": nlp_layers.PositionEmbedding,
        "keras_nlp>TransformerEncoder": nlp_layers.TransformerEncoder,
        "keras_nlp>TransformerDecoder": nlp_layers.TransformerDecoder,
        "Addons>mish": tfa_activations.mish,
    }
    with utils.custom_object_scope(custom_objs):
        model = keras.models.clone_model(base_model)
    return model

model = build_trans()
model.compile(
    loss="MSE",
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=["MSE"]
)
model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=8,
    validation_split=0.2,
)
model.evaluate(x_test, y_test, verbose=1)

Epoch 1/100
20/20 [==============================] - 29s 1s/step - loss: 0.7260 - MSE: 0.7260 - val_loss: 0.6598 - val_MSE: 0.6598
Epoch 2/100
20/20 [==============================] - 26s 1s/step - loss: 0.7253 - MSE: 0.7253 - val_loss: 0.6591 - val_MSE: 0.6591
Epoch 3/100
20/20 [==============================] - 26s 1s/step - loss: 0.7246 - MSE: 0.7246 - val_loss: 0.6586 - val_MSE: 0.6586
Epoch 4/100
20/20 [==============================] - 25s 1s/step - loss: 0.7241 - MSE: 0.7241 - val_loss: 0.6581 - val_MSE: 0.6581
Epoch 5/100
20/20 [==============================] - 25s 1s/step - loss: 0.7237 - MSE: 0.7237 - val_loss: 0.6577 - val_MSE: 0.6577
Epoch 6/100
20/20 [==============================] - 25s 1s/step - loss: 0.7233 - MSE: 0.7233 - val_loss: 0.6573 - val_MSE: 0.6573
Epoch 7/100
20/20 [==============================] - 26s 1s/step - loss: 0.7229 - MSE: 0.7229 - val_loss: 0.6569 - val_MSE: 0.6569
Epoch 8/100
20/20 [==============================] - 24s 1s/step - loss: 0.7225 - M

[0.6872627139091492, 0.6872626543045044]

In [7]:
model.predict(x_test)[0]

2/2 [==============================] - 6s 2s/step


array([[ 0.04510546],
       [-0.04705388],
       [ 0.04510548],
       ...,
       [ 0.04510546],
       [ 0.04510546],
       [ 0.04510546]], dtype=float32)

In [ ]:
model.save("Model.h5")